In [124]:
import math

def eval_conductor_section(fault_current, kf, trip_time):        
    conductor_section_kcmil = fault_current*kf*math.sqrt(trip_time)
    conductor_section_mm = conductor_section_kcmil/1.974
    diameter = (2*math.sqrt(conductor_section_mm/math.pi))/1000 # Diameter should be returned in meters
    return diameter

def calc_max_values(trip_time, ground_ro, gravel_ro, gravel_depth):
    max_current = 0.157/math.sqrt(trip_time)
    correction_factor = 1 - (0.09*(1-ground_ro/gravel_ro
                                   )/(2*gravel_depth+0.09))

    max_touch_voltage = (1000+1.5*correction_factor*gravel_ro)*max_current
    max_step_voltage = (1000+6*correction_factor*gravel_ro)*max_current
    print(correction_factor)
    return max_touch_voltage, max_step_voltage

def eval_grid_resistance(width, height, depth, spacement, ground_ro):
    horizontal_conductors = (height/spacement)+1
    vertical_conductors = (width/spacement) + 1
    total_length = horizontal_conductors*width + vertical_conductors*height
    area = width*height
    equivalent_resistance = ground_ro*(
        (1/total_length)+(1/math.sqrt(20*area))*(
        1 + (1/(1+depth*math.sqrt(20/area)))))
    return equivalent_resistance

def eval_gpr(maximum_current, grid_resistance):
    gpr = maximum_current*grid_resistance
    return gpr

def eval_voltages(ground_rods, width, height, spacement, depth, ro, ig, rods_length, rods_number):
    horizontal_conductors = (height/spacement)+1
    vertical_conductors = (width/spacement) + 1
    total_length = horizontal_conductors*width + vertical_conductors*height
    if not ground_rods:
        lm = total_length+rods_length
    else:
        ind_rod = (rods_length/rods_number)
        lm = total_length + rods_length*(1.55+(1.22*(ind_rod/math.sqrt(math.pow(height, 2)+math.pow(width, 2)))))
    area = width*height
    perimeter = 2*width+2*height
    na = (2*total_length)/perimeter
    nb = math.sqrt(perimeter/(4*math.sqrt(area)))
    # Because we only accept square or retangular grinds, we don't need to compute nc and nd
    n_eq = na*nb
    if ground_rods:
        kii = 1
    else:
        kii = (1/math.pow(2*n_eq,(2/n_eq)))
    

    ki = 0.644 + 0.148*n_eq
    kh = math.sqrt(1+depth)
    km = (1/(2*math.pi))*(math.log(
    (math.pow(spacement,2)/(16*depth*diameter))+(math.pow(spacement+2*depth, 2)/(8*spacement*diameter))-(depth/(4*diameter)))+
                             (kii/kh)*math.log(8/(math.pi*(2*n-1))))
    mesh_voltage = (ro*km*ki*ig)/lm
    
    ls = 0.75*total_length + 0.85*rods_length
    
    ks =(1/math.pi)*((1/(2*depth))+(1/(spacement+depth))+(1/spacement)*(1-math.pow(0.5, n_eq-2)))
    
    step_voltage = (ro*ks*ki*ig)/ls
    return mesh_voltage, step_voltage

    
    
    

In [129]:
print(eval_voltages(True, 70, 70, 7, 0.5, 400, 1908, 150, 20))

(749.0586110802963, 549.1107928258439)


In [32]:
height = 20
width = 50
spacement = 1

In [33]:
horizontal_conductors = (height/spacement)+1
vertical_conductors = (width/spacement) + 1
total_length = horizontal_conductors*width + vertical_conductors*height
area = width*height
perimeter = 2*width+2*height
nb = math.sqrt(perimeter/(4*math.sqrt(area)))
na = (2*total_length)/perimeter

In [78]:
spacement = 7
depth = 0.5
diameter = 0.01
n = 11
kii = 0.57
kh = 1.225

In [131]:
spacement = 10
while spacement > 0.2:
    print('Doing calcs for spacement = {}m'.format(spacement))
    mesh, step = eval_voltages(False, 70, 70, spacement, 0.5, 400, 1908, 320, 10)
    print('Mesh voltage is {}V and Step voltage is {}V'.format(mesh,step))
    spacement = spacement - 0.5

Doing calcs for spacement = 10m
Mesh voltage is 982.9716309938763V and Step voltage is 476.7011591219613V
Doing calcs for spacement = 9.5m
Mesh voltage is 959.8211147475148V and Step voltage is 478.5154281114815V
Doing calcs for spacement = 9.0m
Mesh voltage is 935.7949544914362V and Step voltage is 480.6274696263304V
Doing calcs for spacement = 8.5m
Mesh voltage is 910.8198426508873V and Step voltage is 483.09670109611056V
Doing calcs for spacement = 8.0m
Mesh voltage is 884.8118085427525V and Step voltage is 485.9980548455684V
Doing calcs for spacement = 7.5m
Mesh voltage is 857.6739552808168V and Step voltage is 489.4270701101472V
Doing calcs for spacement = 7.0m
Mesh voltage is 829.2935796749044V and Step voltage is 493.50707203864386V
Doing calcs for spacement = 6.5m
Mesh voltage is 799.5384817733586V and Step voltage is 498.39951629565354V
Doing calcs for spacement = 6.0m
Mesh voltage is 768.2522142153371V and Step voltage is 504.3192797461306V
Doing calcs for spacement = 5.5m
Me

doing calc for spacement = 7m
0.889608517824586
doing calc for spacement = 6.5m
0.8680499747589654
doing calc for spacement = 6.0m
0.844967815399228
doing calc for spacement = 5.5m
0.8201404191026942
doing calc for spacement = 5.0m
0.7932980839200245
doing calc for spacement = 4.5m
0.7641105786950384
doing calc for spacement = 4.0m
0.7321728455785488
doing calc for spacement = 3.5m
0.6969924547655152
doing calc for spacement = 3.0m
0.6579923611366824
doing calc for spacement = 2.5m
0.6145759335412208
doing calc for spacement = 2.0m
0.5664203133974388
doing calc for spacement = 1.5m
0.5146277297754377
doing calc for spacement = 1.0m
0.4663741379799984


In [66]:
(kii/kh)*math.log(8/(math.pi*(2*n-1)))

-0.9817078740236755

In [67]:
math.log((math.pow(spacement,2)/(16*depth*diameter))
                             +(math.pow(spacement+2*depth, 2)/(8*spacement*diameter))-(depth/(4*diameter)))

6.571283042360924